# 🏁 Projeto Final — **Assistente Executivo do CEO** (Multiagentes + Paralelo + Tools)

**Objetivo:** Construir um hub de agentes que gere um **Relatório Executivo** integrando **Marketing**, **Vendas** e **Finanças**, executando análises **em paralelo** e consolidando com um **CEO Planner**.

> Conceitos: `Process.hierarchical`, **FileReadTool** (dados reais), orquestração com contextos.


## 🔑 LLM (Groq) — configuração rápida (OpenAI-compatible)
1. Gere uma chave em https://console.groq.com  
2. Defina as variáveis:
   - `OPENAI_API_KEY`: sua chave Groq
   - `OPENAI_API_BASE`: `https://api.groq.com/openai/v1`


In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_GROQ"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("OPENAI_API_BASE set?", "OPENAI_API_BASE" in os.environ)


## 🔧 Passo 0 — Instalação de dependências


In [ ]:
!pip -q install crewai crewai-tools langchain python-dotenv pandas numpy


## 📂 Passo 1 — Dados de entrada (reais via **FileReadTool**)

Você pode **subir seus próprios arquivos** (TXT/CSV) ou usar os **exemplos prontos** abaixo.

Arquivos sugeridos (exemplos prontos):
- **marketing_report.txt**
- **sales_pipeline.csv**
- **finance_results.csv**

> Dica: Carregue seus próprios relatórios para aproximar do seu contexto real.


In [ ]:
from google.colab import files
import os, pandas as pd

USE_UPLOADS = True  # altere para True se quiser subir arquivos

if USE_UPLOADS:
    print("Envie seus arquivos (TXT/CSV). Se preferir, execute novamente com USE_UPLOADS=False para usar exemplos internos.")
    uploaded = files.upload()
    paths = list(uploaded.keys())
    print("Arquivos carregados:", paths)
else:
    # Caminhos padrão (o instrutor fornecerá os arquivos no notebook)
    print("Usando caminhos padrão (você pode editar abaixo para apontar para seus uploads)")


### Defina os caminhos dos arquivos
- `MARKETING_PATH` — texto com métricas e observações (TXT)
- `SALES_PATH` — pipeline de vendas (CSV)
- `FINANCE_PATH` — resultados financeiros (CSV)


In [ ]:
# Ajuste os caminhos conforme seus uploads (se USE_UPLOADS=True)
MARKETING_PATH = "/content/marketing_report.txt"
SALES_PATH     = "/content/sales_pipeline.csv"
FINANCE_PATH   = "/content/finance_results.csv"

print(MARKETING_PATH, SALES_PATH, FINANCE_PATH)


### (Opcional) Criar arquivos de exemplo dentro do Colab
Execute se você não tiver arquivos próprios.


In [ ]:
# example_marketing = '# Relatório de Marketing — 2025-10-30\n\n## Canais e métricas (últimos 30 dias)\n- LinkedIn: 15 posts, 38.5k impressões, 3.1% CTR, 412 cliques, 128 leads\n- Instagram: 9 posts, 21.3k impressões, 1.8% CTR, 113 cliques, 27 leads\n- Newsletter: 2 envios, 9.4k aberturas (28%), 3.9% CTR, 84 cliques, 19 leads\n- Webinars: 1 evento, 142 inscritos, 97 compareceram, 23 leads\n- Paid Ads (LinkedIn): R$ 5.500 investidos, 1.9% CTR, CPC médio R$ 7,12, 39 leads\n\n## Observações\n- Conteúdos técnicos sobre IA geraram maior engajamento no LinkedIn.\n- Picos de tráfego ocorreram nos dias de webinars e no post "Guia prático de multiagentes com CrewAI".\n- Lead magnet mais baixado: "Checklist de Adoção de IA em 30 dias".\n'
# example_sales = 'deal,stage,amount_brl,probability,owner\nOportunidade A,Descoberta,120000,0.4,Ana\nOportunidade B,Proposta,250000,0.6,Bruno\nOportunidade C,Negociação,180000,0.75,Carla\nOportunidade D,Fechamento,90000,0.9,Daniel\nOportunidade E,Proposta,150000,0.55,Ana\nOportunidade F,Descoberta,200000,0.35,Carla\nOportunidade G,Negociação,300000,0.7,Bruno\n'
# example_finance = 'month,revenue_brl,cost_brl,gross_profit_brl,gross_margin\n2025-05-01,835700,705405,130295,0.1559\n2025-06-01,1061828,594829,466999,0.4398\n2025-07-01,973206,733337,239869,0.2465\n2025-08-01,834378,680131,154247,0.1849\n2025-09-01,880587,481078,399509,0.4537\n2025-10-01,1010591,771955,238636,0.2361\n'

# with open(MARKETING_PATH, "w", encoding="utf-8") as f:
#     f.write(example_marketing)

# with open(SALES_PATH, "w", encoding="utf-8") as f:
#     f.write(example_sales)

# with open(FINANCE_PATH, "w", encoding="utf-8") as f:
#     f.write(example_finance)

# print("Arquivos de exemplo gravados em /content")


## 🧠 Passo 2 — Conceitos principais
- **Agentes especialistas**: Marketing, Vendas e Finanças (cada um lê seu arquivo com **FileReadTool**).  
- **Paralelismo**: as análises rodam ao mesmo tempo (`Process.parallel`).  
- **Agregação (fan-in)**: o **CEO Planner** consolida os resultados em um **Relatório Executivo**.


## ✅ Passo 3 — Imports, Tools e definição dos Agentes


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileReadTool

marketing_tool = FileReadTool(file_path=MARKETING_PATH)
sales_tool     = FileReadTool(file_path=SALES_PATH)
finance_tool   = FileReadTool(file_path=FINANCE_PATH)

marketing_analyst = Agent(
    role="Analista de Marketing",
    goal=("DESCREVA O OBJETIVO"),
    backstory=("DESCREVA O BACKSTORY"),
    tools=[marketing_tool],
    allow_delegation=False,
    verbose=True
)

sales_analyst = Agent(
    role="Analista de Vendas",
    goal=("DESCREVA O OBJETIVO"),
    backstory=("DESCREVA O BACKSTORY"),
    tools=[sales_tool],
    allow_delegation=False,
    verbose=True
)

finance_analyst = Agent(
    role="Analista Financeiro",
    goal=("DESCREVA O OBJETIVO"),
    backstory=("DESCREVA O BACKSTORY"),
    tools=[finance_tool],
    allow_delegation=False,
    verbose=True
)

ceo_planner = Agent(
    role="CEO Planner",
    goal=("Consolidar as análises em um Relatório Executivo com foco educacional: "
          "o que aconteceu, por que importa e o que fazer agora."),
    backstory=("Você é o assistente executivo do CEO, capaz de conectar Marketing, Vendas e Finanças."),
    allow_delegation=False,
    verbose=True
)


## 🗂️ Passo 4 — Tarefas em paralelo + consolidação


In [ ]:
marketing_task = Task(
    description=(
        "Usando a ferramenta de leitura, analise o relatório de marketing e produza:"
        "• 3–5 insights por canal (LinkedIn, Instagram, Newsletter, Webinars, Ads)"
        "• Principais picos e conteúdos que performaram melhor"
        "• 3 oportunidades acionáveis para os próximos 30 dias"
        "Evite jargões e não invente dados que não estejam no arquivo."
    ),
    expected_output="Resumo de marketing em Markdown, com bullets e mini-tabela de canais (se possível).",
    agent=marketing_analyst,
)

sales_task = Task(
    description=(
        "Usando a ferramenta de leitura, analise o pipeline de vendas (CSV) e calcule:"
        "• Soma por estágio"
        "• Weighted pipeline (soma(amount_brl*probability))"
        "• Top 3 maiores deals (valor)"
        "• Gargalos (estágios lentos ou com baixa conversão)"
        "Entregue em Markdown com tabela simples."
    ),
    expected_output="Resumo de vendas em Markdown, incluindo tabela por estágio e weighted pipeline.",
    agent=sales_analyst,
)

finance_task = Task(
    description=(
        "Usando a ferramenta de leitura, analise resultados financeiros (CSV) e produza:"
        "• Receita, custo e margem bruta por mês"
        "• Variações MoM e tendência geral"
        "• Riscos e oportunidades (ex.: custos, sazonalidade)"
        "Entregue em Markdown com uma tabela mensal."
    ),
    expected_output="Resumo financeiro em Markdown, com tabela mensal e leitura executiva.",
    agent=finance_analyst,
)

ceo_report_task = Task(
    description=(
        "Consolide os três resumos (Marketing, Vendas, Finanças) em um **Relatório Executivo** (Markdown) com:"
        "1) Introdução: resumo geral (1 parágrafo)"
        "2) Marketing: 1–2 parágrafos + bullets principais"
        "3) Vendas: 1–2 parágrafos + tabela curta de estágios e weighted pipeline"
        "4) Finanças: 1–2 parágrafos + tabela mensal (resumo)"
        "5) Plano de Ação (30–60 dias): 5–8 itens priorizados (RICE simples: impacto/complexidade)"
        "6) Conclusão: próximos passos e métricas a monitorar"
        "Tom educacional, claro e objetivo."
    ),
    expected_output="Relatório Executivo final em Markdown e em português.",
    agent=ceo_planner,
    context=[marketing_task, sales_task, finance_task],
)


## 🚀 Passo 5 — Orquestração e execução em paralelo


In [ ]:
crew = Crew(
    agents=[marketing_analyst, sales_analyst, finance_analyst, ceo_planner],
    tasks=[marketing_task, sales_task, finance_task, ceo_report_task],
    process=Process.hierarchical,
    #Altere conforme o LLM que estará utilizando
    manager_llm="gpt-4",
    verbose=True
)
print(">>> Executando Projeto Final (paralelo + tools)...")
result = crew.kickoff()


## 👀 Passo 6 — Visualizar o Relatório Executivo (Markdown)


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

## 🔁 Desafios (para nota extra)
1) Substitua os arquivos de exemplo pelos **seus próprios** relatórios reais.  
2) Adicione um agente **Revisor de Consistência** para checar contradições entre áreas.  
3) Inclua um **Analisador de Notícias** (SerperDevTool) e peça ao CEO Planner para considerar manchetes relevantes.  
4) Mude para `Process.sequential` e compare tempo/qualidade.  
5) Gere uma **seção de riscos cruzados** (ex.: queda de leads impacta vendas futuras).


## 🌐 (Opcional) Passo Extra — Notícias do Mercado com **SerperDevTool** (Web Search)

Este passo adiciona um agente que busca **manchetes recentes** para enriquecer o relatório do CEO.  
Você precisará de uma `SERPER_API_KEY` (gratuita para testes) obtida em: https://serper.dev/


In [ ]:
# Instalar dependências (se ainda não instaladas)
!pip -q install crewai-tools


### Configurar a chave do Serper
Defina a variável de ambiente `SERPER_API_KEY` no Colab.


In [ ]:
import os
# os.environ["SERPER_API_KEY"] = "SUA_CHAVE_DA_SERPER_AQUI"

print("SERPER_API_KEY set?", "SERPER_API_KEY" in os.environ)


### Criar o Agente de Notícias e Tarefa de Busca
O agente usará a **SerperDevTool** para buscar 3–5 manchetes relevantes (ex.: "mercado brasileiro", "setor tecnologia", "taxa de juros").  
O resultado será fornecido ao **CEO Planner** como **contexto adicional**.


In [ ]:
from crewai_tools import SerperDevTool

# Tool de busca web
news_tool = SerperDevTool()  # usa SERPER_API_KEY

news_agent = Agent(
    role="Agente de Notícias",
    goal=("Coletar 3–5 manchetes relevantes e recentes que possam impactar Marketing, Vendas ou Finanças."),
    backstory=("Você é um pesquisador conciso. Seu foco é buscar manchetes confiáveis, citar a fonte e sintetizar o impacto potencial."),
    tools=[news_tool],
    allow_delegation=False,
    verbose=True
)

news_task = Task(
    description=(
        "Pesquise 3–5 manchetes relevantes (em PT-BR, se possível) sobre o cenário econômico brasileiro "
        "ou o setor de tecnologia e negócios, indicando **título**, **fonte** e **uma linha de impacto**. "
        "Priorize notícias dos últimos 15 dias."
    ),
    expected_output="Lista em Markdown com 3–5 manchetes (título, fonte, link se disponível) e 1 linha de impacto.",
    agent=news_agent,
)


### Reorquestrar: incluir **news_task** como contexto do **CEO Planner**
Você pode **substituir** a criação do `crew` anterior por este bloco que adiciona o `news_task` no contexto do relatório final.


In [ ]:
crew_with_news = Crew(
    agents=[marketing_analyst, sales_analyst, finance_analyst, news_agent, ceo_planner],
    tasks=[marketing_task, sales_task, finance_task, news_task, ceo_report_task],
    process=Process.parallel,
    verbose=True
)

# Atualiza o contexto do relatório do CEO para incluir notícias
ceo_report_task.context = [marketing_task, sales_task, finance_task, news_task]

print(">>> Executando Projeto Final com Notícias (paralelo + tools)...")
result_news = crew_with_news.kickoff()


### Visualizar o Relatório Executivo com Notícias


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result_news.raw))
